In [4]:
import numpy as np
import wandb
import pandas as pd
from tqdm import tqdm

In [5]:
api = wandb.Api(timeout=600)

In [6]:
runs = api.runs("inchangbaek4907/scenario")
len(runs)

214

In [7]:
runs = list(runs)
runs[:5]

[<Run inchangbaek4907/scenario/re2wd1xx (finished)>,
 <Run inchangbaek4907/scenario/h5m442q9 (finished)>,
 <Run inchangbaek4907/scenario/ak7yr998 (finished)>,
 <Run inchangbaek4907/scenario/mwu8l4yk (finished)>,
 <Run inchangbaek4907/scenario/7ek9ue01 (finished)>]

In [8]:

class RunInfo:
    config: dict
    name: str
    run_object: wandb.apis.public.Run
    
    def __init__(self, config, name, run_object):
        self.config = config
        self.name = name
        self.run_object = run_object

    def __str__(self):
        showing_config = dict()
        showing_keys = ['gpt_model']
        
        for key in showing_keys:
            if key in self.config:
                showing_config[key] = self.config[key]
        
        return f"RunInfo(\n\tname={self.name},\n\tconfig={showing_config}\n)"

In [18]:
run_infos = list()

for run in tqdm(runs):
    if run.state == 'finished' and run.summary['Evaluation/llm_iteration'] < 6:
        run_infos.append(RunInfo(run.config, run.name, run))

len(run_infos)

100%|██████████| 214/214 [00:00<00:00, 438486.10it/s]


10

In [19]:
configs = list()
for run_info in tqdm(run_infos[:]):
    configs.append(run_info.config)
    
df = pd.DataFrame(configs)
df

100%|██████████| 10/10 [00:00<00:00, 44667.77it/s]


,lr,pe,seed,task,DEBUG,GAMMA,is_3d,model,n_eps,n_envs,...,gif_frame_duration,feedback_input_type,n_generation_trials,randomize_map_shape,randomize_start_pos,timestep_chunk_size,bypass_feedback_path,eval_max_board_scans,reward_function_path,eval_randomize_map_shape
0,0.0001,got,5,scenario,True,0.99,False,conv,2,600,...,25,array,5,False,False,-1,None,3,None,False
1,0.0001,tot,4,scenario,True,0.99,False,conv,2,600,...,25,array,5,False,False,-1,None,3,None,False
2,0.0001,cot,1,scenario,True,0.99,False,conv,2,600,...,25,array,5,False,False,-1,None,3,None,False
3,0.0001,cot,6,scenario,True,0.99,False,conv,2,600,...,25,array,5,False,False,-1,None,3,None,False
4,0.0001,cot,2,scenario,True,0.99,False,conv,2,600,...,25,array,5,False,False,-1,None,3,None,False
5,0.0001,cot,5,scenario,True,0.99,False,conv,2,600,...,25,array,5,False,False,-1,None,3,None,False
6,0.0001,tot,5,scenario,True,0.99,False,conv,2,600,...,25,array,5,False,False,-1,None,3,None,False
7,0.0001,tot,5,scenario,True,0.99,False,conv,2,600,...,25,array,5,False,False,-1,None,3,None,False
8,0.0001,got,4,scenario,True,0.99,False,conv,2,600,...,25,array,5,False,False,-1,None,3,None,False
9,0.0001,got,5,scenario,True,0.99,False,conv,2,600,...,25,array,5,False,False,-1,None,3,None,False


In [28]:
commands = list()

def get_config(exp_group, target_character):
    return df[(df['exp_group'] == exp_group) & (df['target_character'] == target_character)].iloc[0]

for index, row in df.iterrows():
    
    config = get_config(row['exp_group'], row['target_character'])
    
    seed = row['seed']
    exp_group = row['exp_group']
    target_character = row['target_character']
    
    total_iterations = config['total_iterations']
    gpt_model = config['gpt_model']
    exp_name = config['exp_name']
    n_self_alignment = config['n_self_alignment']
    evaluator = config['evaluator']
    pe = config['pe']
    
    # if nan
    if pd.isna(n_self_alignment):
        n_self_alignment = 0
    n_self_alignment = int(n_self_alignment)
    

    command = f"bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario task=scenario overwrite=True pe={pe} exp_name={exp_name} gpt_model={gpt_model} total_iterations={total_iterations} seed={seed} evaluator={evaluator} target_character={target_character} n_self_alignment={n_self_alignment}"
    commands.append(command)

commands

['bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario task=scenario overwrite=True pe=got exp_name=chatgpt4o gpt_model=chatgpt-4o total_iterations=6 seed=5 evaluator=hr target_character=1 n_self_alignment=0',
 'bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario task=scenario overwrite=True pe=tot exp_name=chatgpt4osa gpt_model=chatgpt-4o total_iterations=6 seed=4 evaluator=hr target_character=1 n_self_alignment=5',
 'bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario task=scenario overwrite=True pe=cot exp_name=chatgpt4osa gpt_model=chatgpt-4o total_iterations=6 seed=1 evaluator=hr target_character=2 n_self_alignment=5',
 'bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario task=scenario overwrite=True pe=cot exp_name=chatgpt4osa gpt_model=chatgpt-4o total_iterations=6 seed=6 evaluator=hr target_character=1 n_self_alignment=5',
 'bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario task=scenario overwrite=True 

In [29]:
commands_block = " \n".join(commands)
from IPython.display import display, Markdown
display(Markdown("```bash\n" + commands_block + ""))

```bash
bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario task=scenario overwrite=True pe=got exp_name=chatgpt4o gpt_model=chatgpt-4o total_iterations=6 seed=5 evaluator=hr target_character=1 n_self_alignment=0 
bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario task=scenario overwrite=True pe=tot exp_name=chatgpt4osa gpt_model=chatgpt-4o total_iterations=6 seed=4 evaluator=hr target_character=1 n_self_alignment=5 
bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario task=scenario overwrite=True pe=cot exp_name=chatgpt4osa gpt_model=chatgpt-4o total_iterations=6 seed=1 evaluator=hr target_character=2 n_self_alignment=5 
bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario task=scenario overwrite=True pe=cot exp_name=chatgpt4osa gpt_model=chatgpt-4o total_iterations=6 seed=6 evaluator=hr target_character=1 n_self_alignment=5 
bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario task=scenario overwrite=True pe=cot exp_name=chatgpt4osa gpt_model=chatgpt-4o total_iterations=6 seed=2 evaluator=hr target_character=1 n_self_alignment=5 
bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario task=scenario overwrite=True pe=cot exp_name=chatgpt4osa gpt_model=chatgpt-4o total_iterations=6 seed=5 evaluator=hr target_character=1 n_self_alignment=5 
bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario task=scenario overwrite=True pe=tot exp_name=chatgpt4osa gpt_model=chatgpt-4o total_iterations=6 seed=5 evaluator=hr target_character=2 n_self_alignment=5 
bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario task=scenario overwrite=True pe=tot exp_name=chatgpt4osa gpt_model=chatgpt-4o total_iterations=6 seed=5 evaluator=hr target_character=1 n_self_alignment=5 
bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario task=scenario overwrite=True pe=got exp_name=chatgpt4osa gpt_model=chatgpt-4o total_iterations=6 seed=4 evaluator=hr target_character=2 n_self_alignment=5 
bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario task=scenario overwrite=True pe=got exp_name=chatgpt4osa gpt_model=chatgpt-4o total_iterations=6 seed=5 evaluator=hr target_character=2 n_self_alignment=5